In [4]:
# eval() и query() - функции ускоряющие работу с pandas (они не создают промежу-
# точных объектов, сразу выполняя код в Си)

import numpy as np

rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y # довольно быстро, и быстрее чем в чистом питоне

4.46 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
# однако есть нюанс. Вот эта операция:

mask = (x > 0.5) & (y < 0.5)

# на самом деле выполняется вот так:

tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

# т.е. создаются два промежуточных объекта в памяти, что, при больших размерах
# массивов х и у может перегрузить и память и процессор

# для операций с массивами в numpy используй numexpr

In [35]:
# библиотека numexpr решает эту проблему, позволяя выполнять действия поэлементно
# и без создания промежуточных объектов
# её функции принимают строку аналогичную строке которую мы вводим в обычные
# функции numpy

import numexpr

mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)
# allclose Returns True if two arrays are element-wise equal within a tolerance
# у нас True - массивы равнозначны. Однако расчет mask_numexpr был эффективней

True

# pd.eval() и DataFrame.eval()

In [37]:
# pd.eval() and  DataFrame.eval() are conceptually similar, and depend
# on the Numexpr package

import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

# считаем сумму обычным методом pandas
%timeit df1 + df2 + df3 + df4

In [30]:
# считаем сумму через eval()
%timeit pd.eval('df1 + df2 + df3 + df4')

# получаем результат в ДВА раза быстрее, и БЕЗ дополнительного расхода памяти

44.5 ms ± 903 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

# eval() поддерживает все арифметические операции
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

# eval() поддерживает все операции сравнения
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

# eval() поддерживает & и | (это вертикальная черта, не деление (деление вот /)
# она немного наклоненна из-за того что комментарии пишутся с наклоном)
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

# eval() поддерживает and и or
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

# eval() поддерживает доступ к объектам через их атрибуты, используя синтаксис
# object.attribute и доступ через индексы, через object[index]
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

# если нужно что-то помимо этого - читай документацию, т.к. библиотека постоянно
# расширяется и то что тебе нужно уже может в ней быть

True

In [32]:
# помимо pd.eval() - функции pandas, у фреймов есть аналогичный метод df.eval()
# он (как и df.query, см. далее) по сути вызывает pd.eval().

df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

# теперь сложим столбцы и функцией и методом
result1 = (df['A'] + df['B']) / (df['C'] - 1)     # чистый питон
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")   # eval() функция pandas
result3 = df.eval('(A + B) / (C - 1)')            # eval() метод фрейма
print(np.allclose(result1, result2), np.allclose(result2, result3))

# результат один, однако методом пользоваться удобнее - пишем меньше кода
# т.к. мы можем обращаться к столбцам через их имя напрямую - А, в то время как
# при использовании eval() функции pandas через массив+имя - df.A

True True


In [40]:
# df.eval() может создавать новые столбцы фрейма
df.eval('D = (A + B) / C', inplace=True)
df.head()

# и модифицировать существующие столбцы
df.eval('D = (A - B) / C', inplace=True)
df.head()

# также df.eval() может принимать локальные переменные через знак @
column_mean = df.mean(1)
result1 = df['A'] + column_mean       # чистый питон
result2 = df.eval('A + @column_mean') # df.eval()
np.allclose(result1, result2)
# @ - означает ЛОКАЛЬНУЮ переменную, а не ИМЯ СТОЛБЦА
# это знак работает ТОЛЬКО в df.eval(), т.к. pd.eval() имеет доступ только к
# месту имен питона, и может обращаться к локальным переменным напрямую
# (а df.eval() "привязан" к месту имен фрейма, и для выхода за его пределы
# мы и используем @)

True

# DataFrame.query()

In [48]:
# однако метод df.eval() не способен на подобную "фильтрацию":
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

# на неё способен pd.eval() и df.query()
result3 = df.query('A < 0.5 and B < 0.5')
print(np.allclose(result1, result2), np.allclose(result2, result3))

# метод df.query() тоже может работать с глобальными переменными через @
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True True


True

# Мораль

Используй df.eval() и df.query() всегда, для всех расчетов. Для больших массивов данных это будет быстрее (хотя, для работы с маленькими массивами чистый питон, как ни странно, будет чуточку быстрее) и самое главное - эффективнее по части расхода памяти.

Использование этих методов - отличная привычка, т.к. они помогут избежать проблем с большими массивами на реальной работе

df.eval()  - для расчетов

df.query() - для фильрации и выбора данных

По особенностям их работы и применения - читай документацию и StackOverflow